In [1]:
library(tidyverse)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
library(brms)
library(ape)
library(loo)

Loading required package: Rcpp

Loading 'brms' package (version 2.17.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar


This is loo version 2.4.1

- Online documentation and vignettes at mc-stan.org/loo

- As of v2.0.0 loo defaults to 1 core but we recommend using as many as possible. Use the 'cores' argument or set options(mc.cores = NUM_CORES) for an entire session. 



In [3]:
d <- read_csv("../data/data_cues.txt")

Rows: 30 Columns: 7

── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (3): Glottocode, Language, Genus
dbl (4): Case_Marking, Tight_Semantics, Rigid_Order, Verb_Middle


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



In [4]:
head(d)

Glottocode,Language,Genus,Case_Marking,Tight_Semantics,Rigid_Order,Verb_Middle
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
stan1318,Arabic,Semitic,0.036,0.218,0.655,0.391
bulg1262,Bulgarian,Slavic,0.028,0.144,0.782,0.966
croa1245,Croatian,Slavic,0.415,0.147,0.414,0.900
czec1258,Czech,Slavic,0.525,0.172,0.240,0.818
dani1285,Danish,Germanic,0.000,0.208,0.926,0.912
dutc1256,Dutch,Germanic,0.000,0.214,0.817,0.462


In [5]:
d %>%
    mutate(
        x1 = scale(Case_Marking), 
        x2 = scale(Tight_Semantics),
        x3 = scale(Rigid_Order),
        x4 = scale(Verb_Middle)
    ) -> d
head(d)

Glottocode,Language,Genus,Case_Marking,Tight_Semantics,Rigid_Order,Verb_Middle,x1,x2,x3,x4
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>"
stan1318,Arabic,Semitic,0.036,0.218,0.655,0.391,-0.7628649,0.826553846,0.09797316,-0.7625740
bulg1262,Bulgarian,Slavic,0.028,0.144,0.782,0.966,-0.7927616,-0.515764763,0.55542136,0.7687882
croa1245,Croatian,Slavic,0.415,0.147,0.414,0.900,0.6534927,-0.461346441,-0.77009783,0.5930144
czec1258,Czech,Slavic,0.525,0.172,0.240,0.818,1.0645728,-0.007860424,-1.39683789,0.3746289
dani1285,Danish,Germanic,0.000,0.208,0.926,0.912,-0.8974002,0.645159439,1.07410278,0.6249733
dutc1256,Dutch,Germanic,0.000,0.214,0.817,0.462,-0.8974002,0.753996083,0.68148976,-0.5734840


In [73]:
frm = formula(x1 ~ x2 + x3 + x4)

In [74]:
summary(lm(frm, d))


Call:
lm(formula = frm, data = d)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.68439 -0.28572  0.03906  0.32083  1.43754 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -1.565e-16  1.158e-01   0.000   1.0000    
x2           2.193e-01  1.279e-01   1.715   0.0982 .  
x3          -7.346e-01  1.207e-01  -6.085 1.98e-06 ***
x4           1.947e-02  1.277e-01   0.152   0.8800    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6344 on 26 degrees of freedom
Multiple R-squared:  0.6392,	Adjusted R-squared:  0.5976 
F-statistic: 15.35 on 3 and 26 DF,  p-value: 5.99e-06


In [8]:
phylo <- read.nexus("../data/mcc.tre")

In [9]:
A <- 10*ape::vcv.phylo(phylo)

In [75]:
frm = formula(x1 ~ x2 + x3 + x4 + (1|gr(Language, cov = A)))
frm0 = formula(x1 ~ x2 + x3 + x4)

In [76]:
model <- brm(
    frm,
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 1), "b"),
        prior(normal(0, 1), "Intercept"),
        prior(student_t(3, 0, 20), "sd"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=10000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
model <- add_criterion(model, "loo")

Compiling Stan program...

recompiling to avoid crashing R session

Start sampling

Warning message:
“There were 95 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 4036 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“There were 4 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (E

In [77]:
null_model <- brm(
    frm0,
    data=d,
    data2 = list(A = A),
    prior = c(
        prior(normal(0, 1), "b"),
        prior(normal(0, 1), "Intercept"),
        prior(student_t(3, 0, 20), "sigma")
    ),
    iter=10000,
    refresh=0,
    control=list(adapt_delta=0.999),
)
null_model <- add_criterion(null_model, "loo")


Compiling Stan program...

recompiling to avoid crashing R session

Start sampling

Warning message:
“Found 1 observations with a pareto_k > 0.7 in model 'null_model'. It is recommended to set 'moment_match = TRUE' in order to perform moment matching for problematic observations. ”


In [78]:
(mc <- loo_compare(model, null_model))

,elpd_diff,se_diff,elpd_loo,se_elpd_loo,p_loo,se_p_loo,looic,se_looic
model,0.00000,0.000000,-11.39098,3.749701,43.80967,3.659203,22.78196,7.499402
null_model,-22.53138,2.723293,-33.92236,6.256576,6.78274,3.205680,67.84472,12.513152


In [79]:
if (as.numeric(which.min(mc[,"looic"])) == 1) {
    winner <- model
} else {
    winner <- null_model
}

In [80]:
summary(winner)

Warning message:
“There were 95 divergent transitions after warmup. Increasing adapt_delta above 0.999 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: gaussian 
  Links: mu = identity; sigma = identity 
Formula: x1 ~ x2 + x3 + x4 + (1 | gr(Language, cov = A)) 
   Data: d (Number of observations: 30) 
  Draws: 4 chains, each with iter = 10000; warmup = 5000; thin = 1;
         total post-warmup draws = 20000

Group-Level Effects: 
~Language (Number of levels: 30) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.40      0.09     0.19     0.57 1.00      770     1403

Population-Level Effects: 
          Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
Intercept    -0.01      0.26    -0.54     0.50 1.00     5622     6375
x2            0.11      0.13    -0.13     0.37 1.00     5508     9041
x3           -0.72      0.15    -1.01    -0.42 1.00     5778     7340
x4           -0.08      0.17    -0.41     0.26 1.00     5860     6576

Family Specific Parameters: 
      Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sigma     0.21      0.15     0.02     0.60 1.01     

In [81]:
summary(winner)[['fixed']][-1,c(1,3,4)]

Warning message:
“There were 95 divergent transitions after warmup. Increasing adapt_delta above 0.999 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


,Estimate,l-95% CI,u-95% CI
,<dbl>,<dbl>,<dbl>
x2,0.11293134,-0.1348422,0.3667597
x3,-0.71662085,-1.0131689,-0.4182950
x4,-0.08011532,-0.4143219,0.2570393
